# Comparison of different discovery runs
This notebook can be used to compare different discovery test runs.

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os

# Add the path to revcan
sys.path.append("../../..")

from revcan.reverse_engineering.models.car_metadata import Car

## Set file names
Set the names of the files, you want to compare here (Separated with ',').
Check also the output_dir of your files.

In [ ]:
# TODO: Set list of car file names
car_file_names = [
    "model_vin_date-created.json",
    "model_vin_date-created.json",
    "model_vin_date-created.json"
]

# TODO: Check directory of files
output_dir = "../../../data/car_metadata"

## Compare DID Discovery times per server for different test runs

In [ ]:
def format_seconds_to_hms(seconds):
    if np.isnan(seconds):
        return "--:--:--"
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

def load_did_discovery_data_per_server(car_file_names):
    discovery_times = {}
    did_counts = {}
    test_names = {}
    servers_set = set()

    for car_file_name in car_file_names:
        file_path = os.path.abspath(f"{output_dir}/{car_file_name}")
        try:
            car = Car.load(file_path)
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            continue

        test_names[car_file_name] = getattr(car, "test_name", car_file_name)
        server_times = {f"{server.id}": server.did_discovery_time_seconds for server in car.servers if server.did_discovery_time_seconds > 0 and server.discovery_complete_flag == True}
        server_did_counts = {f"{server.id}": len(server.parameters) for server in car.servers if server.did_discovery_time_seconds > 0}

        discovery_times[car_file_name] = server_times
        did_counts[car_file_name] = server_did_counts
        servers_set.update(server_times.keys())
    
    sorted_servers = sorted(list(servers_set))
    num_files = len(car_file_names)
    num_servers = len(sorted_servers)

    # Initialize arrays with NaN (to handle missing values)
    discovery_times_array = np.full((num_files, num_servers), np.nan)
    did_counts_array = np.full((num_files, num_servers), np.nan)

    for i, car_file_name in enumerate(car_file_names):
        for j, server in enumerate(sorted_servers):
            if server in discovery_times.get(car_file_name, {}):
                discovery_times_array[i, j] = discovery_times[car_file_name][server]
            if server in did_counts.get(car_file_name, {}):
                did_counts_array[i, j] = did_counts[car_file_name][server]

    return discovery_times_array, did_counts_array, sorted_servers, test_names

# Load discovery times per server
discovery_times_np, did_counts_np, servers_sorted, test_names = load_did_discovery_data_per_server(car_file_names)

# Plot discovery times per server
plt.figure(figsize=(14, 6))

for i, file_path in enumerate(car_file_names):
    times = discovery_times_np[i]
    plt.plot(servers_sorted, times, marker='o', label=test_names[file_path])

plt.xticks(rotation=45, ha='right')
plt.ylabel('DID Discovery Time (seconds)')
plt.xlabel('Servers')
plt.title('DID Discovery Times Per Server Across Test Runs')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Convert discovery times to hh:mm:ss format
discovery_times_hms = np.vectorize(format_seconds_to_hms)(discovery_times_np)


In [ ]:
discovery_times_df = pd.DataFrame(discovery_times_hms, index=[test_names[file] for file in car_file_names], columns=servers_sorted).T
discovery_times_df.index.name = "Server IDs"
print("\nDiscovery Times Per Server:")
discovery_times_df

In [ ]:
did_counts_df = pd.DataFrame(did_counts_np, index=[test_names[file] for file in car_file_names], columns=servers_sorted).T
did_counts_df.index.name = "Server IDs"
pd.options.display.float_format = '{:,.0f}'.format # Display floats as integers
print("\nNumber of DIDs Found Per Server:")
did_counts_df